In [ ]:
#| default_exp lime

In [ ]:
#| export
from relax.import_essentials import *
from lime.lime_base import LimeBase
from functools import partial
import sklearn

In [ ]:
#| export
class LimeExplanation(object):
    def __init__(self, intercept={}, local_exp={}, score={}, local_pred={}):
        self.intercept = intercept
        self.local_exp = local_exp
        self.score = score
        self.local_pred = local_pred

    def __str__(self):
        return str({
            'intercept': self.intercept,
            'local_exp': self.local_exp,
            'score': self.score,
            'local_pred': self.local_pred
        })

In [ ]:
#| export
class LimeTabularExplainer(object):
    def __init__(self, training_data):
        freq = np.sum(training_data, axis=0)
        freq = freq / len(training_data)
        self.freq = freq
        # kernel_width = None
        kernel_width = np.sqrt(training_data.shape[1]) * .75
        kernel_width = float(kernel_width)

        def kernel(d, kernel_width):
            return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        kernel_fn = partial(kernel, kernel_width=kernel_width)
        self.base = LimeBase(kernel_fn)

    def generate_neighbors(self, x, cat_arrays, cat_idx, num_samples):
        cont_perturbed = x[:, :cat_idx] + np.random.normal(0, 0.1, size=(num_samples, cat_idx))
        cont_perturbed = np.clip(cont_perturbed, 0., 1.)
        _cat_idx = cat_idx
        neighbors = [cont_perturbed]
        for col in cat_arrays:
            cat_end_idx = cat_idx + len(col)
            # one_hot_idx = np.random.randint(0, len(col), size=(num_samples,))
            one_hot_idx = np.random.choice(range(len(col)), size=(num_samples,), p=self.freq[cat_idx: cat_end_idx])
            cat_feat = np.eye(len(col))[one_hot_idx]

            cat_idx = cat_end_idx
            neighbors.append(cat_feat)
        x = x.reshape(1, -1)
        neighbors = np.concatenate(neighbors, axis=-1)
        return np.concatenate((x, neighbors), axis=0)

    def explain_instance(self,
                         x,
                         predict_fn,
                         cat_arrays,
                         cat_idx,
                         labels=(1,),
                         top_labels=None,
                         num_features=200,
                         num_samples=500):
        neighbors = self.generate_neighbors(
            x, cat_arrays=cat_arrays, cat_idx=cat_idx, num_samples=num_samples)
        yss = predict_fn(neighbors) #+ 1e-6
        yss = np.clip(yss, 1e-6, 1. - 1e-6)
        # map to regression model
        yss = - np.log(1 / yss - 1)
        distances = sklearn.metrics.pairwise_distances(
            neighbors, neighbors[0].reshape(1, -1), metric="euclidean"
        ).ravel()

        self.class_names = [str(x) for x in range(yss[0].shape[0])]

        if top_labels:
            labels = np.argsort(yss[0])[-top_labels:]

        intercept, local_exp, score, local_pred = {}, {}, {}, {}
        for label in labels:
            (intercept[label],
             local_exp[label],
             score[label],
             local_pred[label]) = self.base.explain_instance_with_data(
                 neighbors, yss, distances, label, num_features,
                 model_regressor=sklearn.linear_model.Ridge(alpha=1, fit_intercept=False,)
             )
        return LimeExplanation(
            intercept=intercept,
            local_exp=local_exp,
            score=score,
            local_pred=local_pred
        )

In [ ]:
#| export
class LocalApprox(object):
    def __init__(self, train_X, cat_arrays, cat_idx):
        # self.explainer = LimeTabularExplainer(train_X, class_names=['0', '1'], discretize_continuous=False)
        self.explainer = LimeTabularExplainer(train_X)
        self.cat_arrays = cat_arrays
        self.cat_idx = cat_idx

    def extract_weights(self, x_0, pred_fn, shift=0.1):
        # exp = self.explainer.explain_instance(x_0, self.predict_fn, top_labels=1, num_features=200, num_samples=1000)
        exp = self.explainer.explain_instance(
          x_0, pred_fn, self.cat_arrays, self.cat_idx, num_features=200, num_samples=5000)
        coefs = exp.local_exp[1]

        intercept = exp.intercept[1]
        coefs = sorted(coefs, key=lambda x: x[0])

        w = np.array([e[1] for e in coefs])
        return w, intercept #np.array(b).reshape(1,)

## Test

In [ ]:
from cfnet.datasets import TabularDataModule
from sklearn.linear_model import LogisticRegression

dm = TabularDataModule({
    "data_name": "student",
    "continous_cols": [
        "failures", "age"
    ],
    "discret_cols": [
        "G2", "G1", "higher", "goout", "Mjob", "Fjob", "health", 
        "freetime", "absences", "Walc", "famrel", "Medu", "Fedu"
    ],
    "batch_size": 128,
    "data_dir": "assets/data/student/gp.csv"
})
dm.prepare_data()

In [ ]:
X, y = dm.train_dataset[:]

lr = LogisticRegression()
lr.fit(X, y.reshape(-1))

LogisticRegression()

In [ ]:
lr.predict_proba(X[0].reshape(1, -1))

array([[0.03176497, 0.96823503]])

In [ ]:
test_X, _ = dm.test_dataset[:]
lime = LocalApprox(X, cat_arrays=dm.cat_arrays, cat_idx=dm.cat_idx)

for x in test_X:
    x = x.reshape(1, -1)
    coef, intercept = lime.extract_weights(x, lr.predict_proba)
    y_pred = lr.predict(x)
    assert np.abs(1 / (1 + np.exp(- x @ coef)) - lr.predict_proba(x)[0][1]).item() < 0.01
    

In [ ]:
coef.shape

(82,)

In [ ]:
1 / (1 + np.exp(- x @ coef))

array([0.03159808])